# Verify Schema Step Demo
This notebook walks through a simple example of using the verify-schema step in serving functions, by first downloading the step from the hub and inpecting it, then including the step in the function in two different approaches.

## Get the step from the hub

In [2]:
import mlrun

In [3]:
hub_step = mlrun.get_hub_step("hub://verify_schema")

Now the file has been downloaded to our current working directory (provide `local_path` paramater to change this default destination), and we can edit it directly. 

We can also inspect the `hub_stpe` object:

In [5]:
hub_step.to_dict()

{'filename': 'verify_schema.py',
 'example': 'verify_schema.ipynb',
 'local_path': PosixPath('/User'),
 'url': 'hub://verify_schema',
 'class_name': 'VerifySchema',
 'name': 'verify_schema',
 'version': '1.0.0',
 'categories': ['data-preparation', 'model-serving', 'utilities'],
 'description': 'Verifies the event is aligned with the provided schema'}

## Use custom step in a function

When we are done with editing the step code, we are ready to use it in a function, as any other custom step:

In [7]:
project = mlrun.get_or_create_project("verify-schema-step-demo",'./verify-schema-step-demo')

> 2025-12-31 09:00:01,987 [info] Project loaded successfully: {"project_name":"verify-schema-step-demo"}


In [10]:
fn = project.set_function(hub_step.get_src_file_path(), name="serving-fn", kind="serving", image = "mlrun/mlrun" )
graph = fn.set_topology("flow", engine="async")
schema = ["id", "height", "weight"]
graph.to(class_name="VerifySchema", name="verify", schema=schema)

In [9]:
project.deploy_function(fn)

In [11]:
serving_fn = project.get_function("serving-fn")
event = {
    "id": "3425",
    "height": 157,
    "weight": 58
}
serving_fn.invoke("/", body=event)

{'id': '3425', 'height': 157, 'weight': 58}

Or try to raise an error:

In [12]:
event = {
    "id": "3425",
    "height": 157
}
serving_fn.invoke("/", body=event)

## Add step directly from the hub
In case you don't need to make any changes to the step code, you can add the step directly from the hub: 

In [13]:
graph.add_step(class_name="hub://verify_schema", name="verify-2", schema=schema).respond()

In [14]:
project.deploy_function(fn)

In [15]:
event = {
    "id": "3425",
    "height": 157,
    "weight": 58
}
serving_fn.invoke("/", body=event)